# GDP per continent

The task involves analyzing the Gross Domestic Product (GDP) values for each country as listed on the Wikipedia page List of countries by GDP (nominal).
The objective of this analysis is to see whether Europe is the continent with the highest GDP.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)" # URL of the Wikipedia page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'wikitable'}) # Find the table containing the GDP data

gdp_data = []
for row in table.find_all('tr')[1:]: # Extract each row in the table, skipping the header
    cells = row.find_all('td')
    if len(cells) > 3:
        country = cells[0].text.strip()
        if country == "World": # Skip the 'World' row
            continue
        continent = cells[1].text.strip()
        gdp_str = cells[2].text.strip().replace(',', '').replace('$', '')
        try:
            gdp = float(gdp_str)
        except ValueError:
            gdp = 0  # Manage cases where GDP data is missing
        gdp_data.append((country, gdp, continent))

df = pd.DataFrame(gdp_data, columns=['Country', 'GDP', 'Continent']).dropna(subset=['Continent']) # Create a DataFrame from the GDP data
gdp_by_continent = df.groupby('Continent')['GDP'].sum().reset_index() # Aggregate GDP by continent

for _, row in gdp_by_continent.iterrows():  # Print the aggregated GDP by continent
    print(f"{row['Continent']}: {row['GDP'] / 1e6:.2f} million US$")

if not gdp_by_continent.empty:  # Check if Europe has the highest GDP
    highest_gdp_continent = gdp_by_continent.loc[gdp_by_continent['GDP'].idxmax(), 'Continent']
    print("\nContinent with the highest GDP:", highest_gdp_continent)
    print("Has Europe the highest GDP?", highest_gdp_continent == "Europe")
else:
    print("No data available to determine the continent with the highest GDP.")


Africa: 2.82 million US$
Americas: 38.15 million US$
Asia: 36.06 million US$
Europe: 26.45 million US$
Oceania: 2.09 million US$

Continent with the highest GDP: Americas
Has Europe the highest GDP? False
